In [1]:
try:
    import cPickle as pickle
except:
    import pickle
# import plottools
import matplotlib
matplotlib.use('TkAgg')  # Specify the backend
import matplotlib.pyplot as plt
import matplotlib.text
import os
import numpy
import math
import collections
from collections import OrderedDict
import json
import pandas as pd
import matplotlib.gridspec as gridspec

In [2]:
def load_feature_data(path_model_results, path_exp_features, path_feature_categs, features_to_skip=[]):
    """Load model feature data and feature errors, load experimental features and the feature categories"""
    # Loading the model validation results: feature scores and errors
    data_dict = collections.OrderedDict()
    for models in os.listdir(path_model_results):
        data_dict[models] = {}
        with open(os.path.join(path_model_results, models, 'somatic_model_features.json')) as f:
            features = json.load(f, object_pairs_hook=collections.OrderedDict)
            features = OrderedDict((key, value) for key, value in features.items() if key not in features_to_skip)
            data_dict[models]['features'] = collections.OrderedDict(sorted(features.items()))
        with open(os.path.join(path_model_results, models, 'somatic_model_errors.json')) as f:
            errors = json.load(f, object_pairs_hook=collections.OrderedDict)
            errors = OrderedDict((key, value) for key, value in errors.items() if key not in features_to_skip)
            data_dict[models]['errors'] = collections.OrderedDict(sorted(errors.items()))
        with open(os.path.join(path_model_results, models, 'final_score.json')) as f:
            final_score = json.load(f)
            data_dict[models]['final score'] = final_score['score']
    # Loading observation
    with open(path_exp_features) as f:
        observation = json.load(f, object_pairs_hook=collections.OrderedDict)
    # Loading feature categories
    with open(path_feature_categs) as f:
        all_features_dict = json.load(f, object_pairs_hook=collections.OrderedDict)
    feature_cat_names = []
    for key in all_features_dict:
        feature_cat_names.append(key)
    return data_dict, observation, all_features_dict, feature_cat_names

In [3]:
def create_blank_dataframe(feature_cats, models):
    print(feature_cats)
    print(models)
    dataframe = pd.DataFrame(index=models, columns=feature_cats + ["color", "final score", "NaN values"])
    return dataframe

In [4]:
def add_morph_type(dataframe, models, point=[], stylized=[], detailed=[]):
    """Add the type of the model morphology to the dataframe"""
    for model in models:
        #print(model)
        if model in point:
            dataframe.loc[model, "morphology"] = "point"
        elif model in stylized:
            dataframe.loc[model, "morphology"] = "stylized"
        elif model in detailed:
            dataframe.loc[model, "morphology"] = "detailed"
        else:
            dataframe.loc[model, "morphology"] = None
            print("No information about the morphology type of the model", model)


In [5]:
def add_new_model_names(dataframe, new_names_dict):
    for model_name in new_names_dict:
        dataframe.loc[model_name, "new names"] = new_names_dict[model_name]
                      

In [6]:
def add_model_colors(dataframe, models, path_color_file):
    with open(path_color_file, "r") as f:
        model_colors_dict = json.load(f)
    f.close()
    dataframe['color'] = dataframe['color'].astype('object')
    for model in models:
        try:
            dataframe.at[model, "color"] = model_colors_dict[dataframe.loc[model, "new names"]]
        except:
            print("no color info about", model)
            dataframe.at[model, "color"] = None

In [7]:
def plot_category_feat_error_means(data_dict, observation, all_features_dict, feature_cat, dataframe, morph_type, save_dir,  
                                   zoomed=False, xlim_values=[], feature_cat_names_final=None):
    """Calculate feature error means for the features in the category, plot and save the fig, also fills the dataframe"""
    features_in_cat_list = all_features_dict[feature_cat]
    # keep only those features that are present in observation
    features_in_cat_list = [feat for feat in features_in_cat_list if len(list(s for s in list(observation.keys())
                                                                               if feat in s)) != 0]
    plt.clf()
    #plt.style.use("seaborn-whitegrid")
    #plt.style.use('dark_background')
    fig = plt.figure(1)
    ax = fig.add_subplot(111)
    matplotlib.rcParams['axes.spines.right'] = False
    matplotlib.rcParams['axes.spines.top'] = False
    matplotlib.rcParams['axes.spines.left'] = False
    matplotlib.rcParams['axes.spines.bottom'] = False
    try:
        colormap = plt.cm.spectral  # http://matplotlib.org/1.2.1/examples/pylab_examples/show_colormaps.html
    except:
        colormap = plt.cm.nipy_spectral
    #plt.gca().set_prop_cycle(plt.cycler('color', colormap(numpy.linspace(0.1, 0.9, len(list(data_dict.keys()))))))
    plt.gca().set_prop_cycle(plt.cycler('color', colormap(numpy.linspace(0.1, 0.9, 9))))
    #plt.gcf().set_size_inches(210 / 25.4, (297 / 25.4) / 2)

    if morph_type == "point":
        marker_type = 'o'
        marker_size = 10
    elif morph_type == "stylized":
        marker_type = '^'
        marker_size = 10
    elif morph_type == "detailed":
        marker_type = '*'
        marker_size = 15
    else:
        marker_type = ""
        marker_size = 0

        
    for key in list(dataframe.index):
        x = []
        all_nan_values = 0
        for feat in features_in_cat_list:
            feat_vals = list(v for k, v in data_dict[key]['errors'].items() if k.startswith(feat+"."))
            # x.append(numpy.mean(feat_vals))
            #nan_values = sum(math.isnan(x) for x in feat_vals)
            #all_nan_values = all_nan_values + nan_values
            x.append(numpy.nanmean(feat_vals))              # !!!! itt melyik a helyes ???
        y = range(len(features_in_cat_list))
        ax.plot(x, y, marker=marker_type, markersize=marker_size, linestyle='none',  clip_on=False, 
                label=dataframe.loc[key, "new names"], color = dataframe.loc[key, "color"], alpha=0.9)
        # print("x", x)
        dataframe.loc[key, feature_cat] = numpy.nanmean(x)
        #dataframe.loc[key, "NaN values"] = all_nan_values + dataframe.loc[key, "NaN values"]
        #dataframe.loc[key, "final score"] = data_dict[key]["final score"]

    fig.set_figheight(len(features_in_cat_list) / 2)

    lgd = ax.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left', fontsize=15, markerscale=1, frameon=False)

    ax.yaxis.set_ticks(range(len(features_in_cat_list)))
    ax.set_yticklabels(features_in_cat_list, fontsize=15)
    ax.set_ylim(-1, len(features_in_cat_list))
    ax.set_xlabel('Error Scores', fontsize=15, labelpad=10)
    ax.set_ylabel('Features in the Category', fontsize=15, labelpad=10)
    feature_cat_name = feature_cat_names_final[feature_cat]
    if morph_type == 'point':
        morph_type_name = 'Point'
    elif morph_type == 'stylized':
        morph_type_name = 'Stylized'
    elif morph_type == 'detailed':
        morph_type_name = 'Detailed'
    ax.set_title(feature_cat_name + ' Category Feature Errors ' + morph_type_name + ' Models', fontsize=20, pad=20)
    plt.xticks(fontsize=15)
    plt.grid(True, color='lightgrey', linewidth=1)
    plt.xlim(0)
    plt.tick_params(top=False, bottom=False, left=False, right=False)
    # plt.tick_params(labelsize=11)
    matplotlib.rc('axes',edgecolor='lightgrey')
    fig_name = feature_cat + '_feature_errors_means_' + morph_type + '_RAT'
    save_plot_as_svg = os.path.join(save_dir, f"{fig_name}.svg")
    plt.savefig(save_plot_as_svg, bbox_extra_artists=(lgd,), bbox_inches='tight', dpi=600, format='svg')
    # Zoomed in figures
    if zoomed:
        xlim_value_1 = xlim_values[feature_cat_names.index(feat_cat)][0]
        xlim_value_2 = xlim_values[feature_cat_names.index(feat_cat)][1]
        if xlim_value_1 is not None and xlim_value_2 is not None:
            ax.set_xlim(xlim_value_1,xlim_value_2)
            ax.set_title(feature_cat_name + ' Category Feature Errors ' + morph_type_name + ' models ZOOM', fontsize=17, pad=10)
            fig_name = feature_cat + '_feature_errors_means_' + morph_type + '_ZOOM_RAT'
            save_plot_as_svg = os.path.join(save_dir, f"{fig_name}.svg")
            plt.savefig(save_plot_as_svg, bbox_extra_artists=(lgd,), bbox_inches='tight', dpi=600, format='svg')
    return dataframe

In [8]:
def plot_all_cat_errors(dataframe, feature_cats, models, morph_type, save_dir, zoomed=False, xlim_values=[], feature_cat_names_final=None):
    """Plotting the category errors of the models"""

    plt.clf()
    #plt.style.use('dark_background')
    fig = plt.figure(1)
    ax = fig.add_subplot(111)
    try:
        colormap = plt.cm.spectral  # http://matplotlib.org/1.2.1/examples/pylab_examples/show_colormaps.html
    except:
        colormap = plt.cm.nipy_spectral
    # plt.gca().set_prop_cycle(plt.cycler('color', colormap(numpy.linspace(0.1, 0.9, len(list(data_dict.keys()))))))
    plt.gca().set_prop_cycle(plt.cycler('color', colormap(numpy.linspace(0.1, 0.9, 9))))
    # plt.gcf().set_size_inches(230 / 25.4, (297 / 25.4) / 2)

    if morph_type == "point":
        marker_type = 'o'
        marker_size = 10
    elif morph_type == "stylized":
        marker_type = '^'
        marker_size = 10
    elif morph_type == "detailed":
        marker_type = '*'
        marker_size = 15
    else:
        marker_type = ""

    shift_value = 0.025
    feature_categories = list(feature_cat_names_final.values())
    y_indices_original = numpy.arange(1, len(feature_categories)+1)
    print(y_indices_original)
    y_indices = y_indices_original - 0.025 * len(dataframe.columns)/2
    print(y_indices)
    for i, model in enumerate(dataframe.columns):
        # Apply a specific shift based on the model index
        shift = i * shift_value
        y_positions = y_indices + shift
        ax.plot(dataframe[model][0:len(feature_categories)], y_positions, 
                marker=marker_type, markersize=marker_size, linestyle='none', 
                label=dataframe.loc["new names", model], color=dataframe.loc["color", model],
                alpha=0.9)

    plt.xlim(0)
    fig.set_figheight(len(feature_cats))
    ax.set_yticks(y_indices_original)
    ax.set_yticklabels(feature_categories)
    y_indices_original = numpy.append(y_indices_original, y_indices_original[-1]+1)
    for y in (y_indices_original - 0.5):
        ax.hlines(y=y, xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1], color='lightgrey', linewidth=1)

    lgd = ax.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left', fontsize=15, markerscale=1, frameon=False)

    if morph_type == 'point':
        morph_type_name = 'Point'
    elif morph_type == 'stylized':
        morph_type_name = 'Stylized'
    elif morph_type == 'detailed':
        morph_type_name = 'Detailed'
    ax.set_xlabel('Error Scores', fontsize=15, labelpad=10)
    ax.set_ylabel('Feature Categories', fontsize=15, labelpad=10)
    # ax.set_title('Category Error Scores of ' + morph_type_name + ' Models', fontsize=20, pad=10)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    # plt.xticks(rotation=45)
    plt.tick_params(top=False, bottom=False, left=False, right=False)
    ax.grid(True, color='lightgrey', linewidth=1, axis='x')
    matplotlib.rc('axes',edgecolor='lightgrey')
    fig_name = 'Feature_Category_Errors_' + morph_type + '_RAT'
    save_plot_as_svg = os.path.join(save_dir, f"{fig_name}.svg")
    plt.savefig(save_plot_as_svg, bbox_extra_artists=(lgd,), bbox_inches='tight', dpi=600, format='svg')
    # plt.show()
    # Zoomed in figure
    if zoomed:
        xlim_value_1 = xlim_values[0]
        xlim_value_2 = xlim_values[1]
        if xlim_value_1 is not None and xlim_value_2 is not None:
            ax.set_xlim(xlim_value_1,xlim_value_2)
            for y in (y_indices_original - 0.5):
                ax.hlines(y=y, xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1], color='lightgrey', linewidth=1)
            fig_name = 'Feature_Category_Errors_' + morph_type + '_ZOOM_RAT'
            save_plot_as_svg = os.path.join(save_dir, f"{fig_name}.svg")
            plt.savefig(save_plot_as_svg, bbox_extra_artists=(lgd,), bbox_inches='tight', dpi=600, format='svg')


In [9]:
# path to model soma test validation results
path_model_results = os.path.abspath('/home/szabobogi/hippounit_standard_features/validation_results_RAT/results/somaticfeat/')
# path to experimental standard features fot validation 
path_exp_features = os.path.abspath('/home/szabobogi/BluePyEfe_standard_features/BluePyEfe/FINAL_PV_PTI_data/features_for_validation/standard_features_for_validation_RAT_FINAL.json')
# path to the somatic feature categories file 
path_feature_categs = os.path.abspath('/home/szabobogi/hippounit_standard_features/visualization/config/somatic_feature_chategories.json')
# path to model colors file
path_color_file = os.path.abspath('/home/szabobogi/hippounit_standard_features/visualization/config/modelcolors_vibrant_final.json')
# path to save directory
save_dir = os.path.abspath('/home/szabobogi/hippounit_standard_features/visualization/figures_RAT/Feature_categories_comparison_figures/')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [10]:
# load the data
data_dict, observation, all_features_dict, feature_cat_names = load_feature_data(path_model_results, path_exp_features, path_feature_categs)

In [11]:
# create blank dataframe
model_names = list(data_dict.keys())
df_cat_errors = create_blank_dataframe(feature_cat_names, model_names)

['AP_shape', 'AHP', 'adaptation', 'pattern', 'subthreshold']
['Tzilivaki1_2019_0.1_dt', 'Lee_2014', 'Tzilivaki5_2019', 'Cuts&Poir_2015', 'Tzilivaki2_2019', 'Tzilivaki2_2019_0.1_dt', 'Turi_2019', 'Bezaire_2016', 'Tzilivaki4_2019_0.1_dt', 'Vladimirov_2013', 'Stacey(vd)_2009', 'Tzilivaki4_2019', 'Sherif_2020', 'Hu(rb)_2018', 'Saraga_2006_6.3_cels', 'Tzilivaki3_2019_0.1_dt', 'Saudargiene_2015', 'Chiovini_2014', 'Migliore_2018', 'Hummos_2014', 'Neymotin_2013', 'Neymotin_2011', 'Hu(vd)_2018', 'Stacey(rb)_2009', 'Tzilivaki5_2019_0.1_dt', 'Ferr&Asco_2015', 'CA1_Bezaire_2016_(Lee)_standard', 'Tzilivaki3_2019']


In [12]:
# add the morphology types of the models to the dataframe
point_models = ["Neymotin_2011", "Neymotin_2013", "Stacey(rb)_2009", "Stacey(vd)_2009", "Vladimirov_2013", "Hu(rb)_2018", "Hummos_2014"]
stylized_models = ["Cuts&Poir_2015", "Lee_2014", "Turi_2019", "Saudargiene_2015", "Bezaire_2016"]
detailed_models = ["Saraga_2006_6.3_cels", "Tzilivaki1_2019_0.1_dt", "Tzilivaki4_2019_0.1_dt", "Ferr&Asco_2015", "Migliore_2018", "Tzilivaki3_2019_0.1_dt", "Tzilivaki2_2019_0.1_dt", "Tzilivaki5_2019_0.1_dt", "Chiovini_2014"]
add_morph_type(df_cat_errors, model_names, point=point_models, stylized = stylized_models, detailed=detailed_models)
#display(df_cat_errors)

No information about the morphology type of the model Tzilivaki5_2019
No information about the morphology type of the model Tzilivaki2_2019
No information about the morphology type of the model Tzilivaki4_2019
No information about the morphology type of the model Sherif_2020
No information about the morphology type of the model Hu(vd)_2018
No information about the morphology type of the model CA1_Bezaire_2016_(Lee)_standard
No information about the morphology type of the model Tzilivaki3_2019


In [13]:
#display(data_dict)

In [14]:
new_names_dict = {
    "Neymotin_2011" : "Neymotin 2011",
    "Neymotin_2013" : "Neymotin 2013",
    "Stacey(rb)_2009" : "Stacey(rb) 2009", 
    "Stacey(vd)_2009" : "Stacey(vd) 2009", 
    "Vladimirov_2013" : "Vladimirov 2013", 
    "Hu(rb)_2018" : "Hu(rb) 2018", 
    #"Hu(vd)_2018" : "Hu(vd) 2018",
    #"Sherif_2020" : "Sherif 2020",
    "Hummos_2014" : "Hummos 2014", 
    "Cuts&Poir_2015" : "Cuts&Poir 2015",
    "Lee_2014" : "Lee 2014",
    "Turi_2019" : "Turi 2019",
    "Saudargiene_2015" : "Saudargiene 2015",
    "Bezaire_2016" : "Bezaire 2016",
    "Ferr&Asco_2015" : "Ferr&Asco 2015", 
    "Migliore_2018" : "Migliore 2018", 
    "Saraga_2006_6.3_cels" : "Saraga 2006 6.3C", 
    "Chiovini_2014" : "Chiovini 2014",
    "Tzilivaki1_2019" : "Tzilivaki.1 2019 no set dt", 
    "Tzilivaki2_2019" : "Tzilivaki.2 2019 no set dt", 
    "Tzilivaki3_2019" : "Tzilivaki.3 2019 no set dt", 
    "Tzilivaki4_2019" : "Tzilivaki.4 2019 no set dt",
    "Tzilivaki5_2019" : "Tzilivaki.5 2019 no set dt", 
    "Tzilivaki1_2019_0.1_dt" : "Tzilivaki.1 2019",
    "Tzilivaki2_2019_0.1_dt" : "Tzilivaki.2 2019",
    "Tzilivaki3_2019_0.1_dt" : "Tzilivaki.3 2019",
    "Tzilivaki4_2019_0.1_dt" : "Tzilivaki.4 2019",
    "Tzilivaki5_2019_0.1_dt" : "Tzilivaki.5 2019"
}

# Adding new more sophisticated names to the models in a new colunm
add_new_model_names(df_cat_errors, new_names_dict)
# Sorting the models in alphabetical order in the table
df_cat_errors.sort_index(key=lambda x: x.str.lower(), inplace = True)
#display(df_cat_errors)

In [15]:
add_model_colors(df_cat_errors, model_names, path_color_file)

no color info about Tzilivaki5_2019
no color info about Tzilivaki2_2019
no color info about Tzilivaki4_2019
no color info about Sherif_2020
no color info about Hu(vd)_2018
no color info about CA1_Bezaire_2016_(Lee)_standard
no color info about Tzilivaki3_2019


In [16]:
#display(df_cat_errors)

In [17]:
feature_cat_names_final = {
    'AP_shape': 'AP Shape',
    'AHP': 'AHP',
    'adaptation': 'Adaptation',
    'pattern': 'Firing Pattern',
    'subthreshold': 'Subthreshold'
}
feature_cat_names_final = OrderedDict(feature_cat_names_final)
print(list(feature_cat_names_final.values()))

['AP Shape', 'AHP', 'Adaptation', 'Firing Pattern', 'Subthreshold']


In [18]:
# plot the feature errors in the feature categories and fill the dataframe with the category errors of the point models
pd.set_option('mode.chained_assignment', None)
morph = "point"
# AP_shape AHP adaptation pattern subthreshold  a feature sorrend a zoomhoz
zoomed = False
xlim_values = [[0,4], [None,None], [None,None], [None,None], [None, None], [None, None]]
savedir_morph = os.path.abspath(os.path.join(save_dir, "Point_models"))
if not os.path.exists(savedir_morph):
    os.makedirs(savedir_morph)
df_cat_errors_filtered = df_cat_errors[df_cat_errors['morphology'] == morph]
for feat_cat in feature_cat_names:
    df_cat_errors_filtered = plot_category_feat_error_means(data_dict, observation, all_features_dict, feat_cat, 
                                                            df_cat_errors_filtered,  morph, savedir_morph, zoomed, 
                                                            xlim_values, feature_cat_names_final)
# plot the category errors of the point models
zoomed = True
xlim_values = [0, 10]
plot_all_cat_errors(df_cat_errors_filtered.T, feature_cat_names, model_names, morph, savedir_morph, 
                    zoomed, xlim_values, feature_cat_names_final)
matplotlib.pyplot.close()

[1 2 3 4 5]
[0.9125 1.9125 2.9125 3.9125 4.9125]


In [19]:
# plot the feature errors in the feature categories and fill the dataframe with the category errors of the stylized models
morph = "stylized"
# AP_shape AHP adaptation pattern subthreshold   a feature sorrend
#zoomed = True
#xlim_values = [[None,None], [None,None], [None,None], [0,4], [None, None], [None, None]]
savedir_morph = os.path.abspath(os.path.join(save_dir, "Stylized_models"))
if not os.path.exists(savedir_morph):
    os.makedirs(savedir_morph)
df_cat_errors_filtered = df_cat_errors[df_cat_errors['morphology'] == morph]
for feat_cat in feature_cat_names:
    df_cat_errors_filtered = plot_category_feat_error_means(data_dict, observation, all_features_dict, feat_cat, 
                                                            df_cat_errors_filtered,  morph, savedir_morph, 
                                                            feature_cat_names_final=feature_cat_names_final)
# plot the category errors of the point models
zoomed = True
xlim_values = [0, 10]
plot_all_cat_errors(df_cat_errors_filtered.T, feature_cat_names, model_names, morph, savedir_morph, 
                    zoomed, xlim_values, feature_cat_names_final)
matplotlib.pyplot.close()

[1 2 3 4 5]
[0.9375 1.9375 2.9375 3.9375 4.9375]


In [20]:
# plot the feature errors in the feature categories and fill the dataframe with the category errors of the detailed models
morph = "detailed"
# AP_shape AHP adaptation pattern subthreshold   a feature sorrend
zoomed = True
xlim_values = [[0,30], [None,None], [0, 3], [0,10], [None, None], [None, None]]
savedir_morph = os.path.abspath(os.path.join(save_dir, "Detailed_models"))
if not os.path.exists(savedir_morph):
    os.makedirs(savedir_morph)
df_cat_errors_filtered = df_cat_errors[df_cat_errors['morphology'] == morph]
for feat_cat in feature_cat_names:
    df_cat_errors_filtered = plot_category_feat_error_means(data_dict, observation, all_features_dict, feat_cat, 
                                                            df_cat_errors_filtered,  morph, savedir_morph, zoomed, 
                                                            xlim_values, feature_cat_names_final)
# plot the category errors of the point models
zoomed = True
xlim_values = [0, 10]
plot_all_cat_errors(df_cat_errors_filtered.T, feature_cat_names, model_names, morph, savedir_morph, 
                    zoomed, xlim_values, feature_cat_names_final)
matplotlib.pyplot.close()

/tmp/ipykernel_1507106/3375294699.py:47: RuntimeWarning: Mean of empty slice
  x.append(numpy.nanmean(feat_vals))              # !!!! itt melyik a helyes ???


[1 2 3 4 5]
[0.8875 1.8875 2.8875 3.8875 4.8875]


In [21]:
# set xlim_vales for the category feature errors figures if needed:
#zoomed = True
#xlim_values = [[None,None], [None,None], [None, None], [0,3], [None, None], [None, None]]

# set xlim_vales for the ALL_feat_cats_mean_errors figure if needed:
#zoomed = True
#xlim_values = [0, 1]
#plot_all_cat_errors(df_cat_errors.T, feature_cat_names, model_names, savedir, zoomed, xlim_values)

In [22]:
# Kövi napirendi pontok:

# a megfelelő modell színek véglegesítése  
# - tesztelés colorwheel-el ? külön ciklus a pont stylized és detailed modellekre a hosszuktol függyően ?

# Edin különböző hőmérsékletekkel is felkerülhet !

# dendrites és axonos összehasonlító ábra készítése 


# ábrák egységesítése - modellek színei, nevei ujnevei mindenhol meglegyenek - 

# axon és bAP comparison ábrákon is 
# feature chategories plotok fehérben is meglegyenek

# modell neven uj nevek és színek mindenhol egységeseklegyenek !!! 







In [23]:
display(df_cat_errors_filtered)

,AP_shape,AHP,adaptation,pattern,subthreshold,color,final score,NaN values,morphology,new names
Chiovini_2014,1.15515,0.812721,2.106888,3.158669,0.915973,"[0.7, 0.271, 0.149, 1.0]",NaN,NaN,detailed,Chiovini 2014
Ferr&Asco_2015,4.303734,4.149281,3.51837,3.503352,0.732357,"[1.0, 0.4279414544120426, 0.0, 1.0]",NaN,NaN,detailed,Ferr&Asco 2015
Migliore_2018,3.121233,1.845305,3.735485,3.682326,1.533634,"[0.772, 0.8, 0.0, 1.0]",NaN,NaN,detailed,Migliore 2018
Saraga_2006_6.3_cels,4.381691,2.790543,2.347306,22.719396,5.276114,"[0.0, 0.6, 0.0, 1.0]",NaN,NaN,detailed,Saraga 2006 6.3C
Tzilivaki1_2019_0.1_dt,6.992732,2.119659,1.891277,2.088171,3.228138,"[0.0, 1.0, 0.6709552500401159, 1.0]",NaN,NaN,detailed,Tzilivaki.1 2019
Tzilivaki2_2019_0.1_dt,6.790114,2.555984,1.833998,2.382359,1.681825,"[0.0, 0.5647084533113945, 1.0, 1.0]",NaN,NaN,detailed,Tzilivaki.2 2019
Tzilivaki3_2019_0.1_dt,6.904819,2.340327,1.839517,2.480444,2.418463,"[0.00963054632172239, 0.0, 1.0, 1.0]",NaN,NaN,detailed,Tzilivaki.3 2019
Tzilivaki4_2019_0.1_dt,7.954745,2.816526,1.880142,2.452709,1.249009,"[0.8639695459548403, 0.0, 1.0, 1.0]",NaN,NaN,detailed,Tzilivaki.4 2019
Tzilivaki5_2019_0.1_dt,7.713695,2.55046,1.858578,2.472653,1.38629,"[1.0, 0.0, 0.22169145441204255, 1.0]",NaN,NaN,detailed,Tzilivaki.5 2019
